In [2]:
import csv
import pandas as pd
import re

book = pd.ExcelFile('./raw/H28_D_working.xlsx')
basic_columns=["分類","項目名","blank","H28対応項目","削除項目","追加項目","メモ"]
knowledge=pd.DataFrame([],columns=["臓器",*basic_columns])

ignore_sheets=["新35症候等","D担当・分析V7最終",'D第4層本文']
sheet_names=list(filter(lambda x:not x in ignore_sheets,book.sheet_names))
for name in sheet_names:
    df = book.parse(name,skiprows=7)
    organ_name = re.sub(r'^.+[ ．\.]',"",book.parse(name,nrows=1,header=None).at[0,0])
    df.columns=basic_columns
    df= df.dropna(subset=["項目名"])
    df["分類"]=df["分類"].fillna(method="ffill")
    df.insert(loc=0,column="臓器",value=organ_name)
    df=df.fillna("")
    knowledge=pd.concat([knowledge,df],axis=0)

knowledge["分類"]=knowledge["分類"].str.replace(r'[\d\(\))（）]','', regex=True)
for name in ["メモ","追加項目","H28対応項目","削除項目"]:
    knowledge[name]=knowledge[name].str.replace(r'\n|\r',',', regex=True)
knowledge["項目名"]=knowledge["項目名"].str.replace(r'[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉞㉕㉖㉗㉘㉙㉚➓]','', regex=True)
knowledge["項目名"]=knowledge["項目名"].str.replace(r'[\(（]\d\d?[\)）]','', regex=True)


knowledge.loc[knowledge["分類"]=="症状・症候","項目名"]=knowledge[knowledge["分類"]=="症状・症候"]["項目名"].str.split("、")
knowledge=knowledge.explode("項目名")
knowledge=knowledge.drop("blank",axis=1)
knowledge.to_csv("./dist/draft_knowlege/knowledge_table.csv",encoding="utf_8_sig",quoting=csv.QUOTE_NONNUMERIC,index=False)
knowledge

,臓器,分類,項目名,H28対応項目,削除項目,追加項目,メモ
0,血液・造血器・リンパ系,構造と機能,骨髄の構造,D-01-01-na-01,,,
1,血液・造血器・リンパ系,構造と機能,造血幹細胞から各血球への分化と成熟の過程,D-01-01-na-02,,,
2,血液・造血器・リンパ系,構造と機能,主な造血因子（エリスロポエチン、顆粒球コロニー刺激因子<G-CSF>、トロンボポエチン）,D-01-01-na-03,,,
3,血液・造血器・リンパ系,構造と機能,脾臓、胸腺、リンパ節、扁桃とPeyer 板の構造と機能,D-01-01-na-04,,,
4,血液・造血器・リンパ系,構造と機能,血漿タンパク質の種類と機能,D-01-01-na-05,,,
...,...,...,...,...,...,...,...
21,精神系,疾患・病態,不安障害（パニック障害・社交不安障害）,D-15-03-na-07,,,
22,精神系,疾患・病態,解離性障害,D-15-03-na-09,,,
23,精神系,疾患・病態,身体表現性障害（身体化障害・疼痛性障害・心気症）、心身症、ストレス関連障害（急性ストレス障害...,"D-15-03-na-07, D-15-03-na-08",,身体化障害,
24,精神系,疾患・病態,摂食障害（神経性食思不振症・神経性過食症）,D-15-03-na-08,,神経性食思不振症、神経性過食症,


In [70]:
book = pd.ExcelFile('./raw/H28_D_working.xlsx')
knowledge_items=pd.DataFrame([],columns=["臓器","第4層","分類"])

ignore_sheets=["新35症候等","D担当・分析V7最終",'D第4層本文']
sheet_names=list(filter(lambda x:not x in ignore_sheets,book.sheet_names))
for name in sheet_names:
    organ_name = re.sub(r'^.+[ ．\.]',"",book.parse(name,nrows=1,header=None).at[0,0])
    item = book.parse(name,nrows=5,skiprows=2,header=None).loc[:,[0]]
    item.columns = ["第4層"]
    item["分類"]=["構造と機能","症状・症候","検査方法","特異的治療法","疾患・病態"]
    item.insert(loc=0,column="臓器",value=organ_name)

    knowledge_items=pd.concat([knowledge_items,item],axis=0)

knowledge_items["第4層"]=knowledge_items["第4層"].str.replace(r'^.+[　\.．]',"",regex=True)
knowledge_items["第4層"]=knowledge_items["第4層"].str.replace(r'(?<!。)$',"。",regex=True)
knowledge_items=pd.merge(knowledge_items,knowledge,how="outer",on=["分類","臓器"])
knowledge_items.fillna("")
knowledge_items=knowledge_items.groupby(["臓器","第4層","分類"],as_index=False,sort=False).agg(lambda x:",".join(x))
knowledge_items["第2層"]="人体各器官の正常構造と機能、病態、診断、治療"
knowledge_items=knowledge_items.loc[:,["第2層","臓器","第4層","H28対応項目"]].rename(columns={"臓器":"第3層"})
knowledge_items.to_csv("./dist/draft_knowlege/knowledge_items.csv",encoding="utf_8_sig",quoting=csv.QUOTE_NONNUMERIC,index=False)
knowledge_items

,第2層,第3層,第4層,H28対応項目
0,人体各器官の正常構造と機能、病態、診断、治療,血液・造血器・リンパ系,血液・造血器・リンパ系の構造と機能について基本的事項を説明できる。,"D-01-01-na-01,D-01-01-na-02,D-01-01-na-03,D-01..."
1,人体各器官の正常構造と機能、病態、診断、治療,血液・造血器・リンパ系,血液・造血器・リンパ系でみられる症状・症候について説明できる。,"D-01-03-na-01,D-01-03-na-02, D-01-03-na-03, D-..."
2,人体各器官の正常構造と機能、病態、診断、治療,血液・造血器・リンパ系,血液・造血器・リンパ系で行う検査方法について基本的事項を説明できる。,"D-01-02-na-01,なし,D-01-02-na-02,なし,D-01-02-na-0..."
3,人体各器官の正常構造と機能、病態、診断、治療,血液・造血器・リンパ系,血液・造血器・リンパ系疾患に特異的な治療法について基本的事項を説明できる。,"なし,なし"
4,人体各器官の正常構造と機能、病態、診断、治療,血液・造血器・リンパ系,血液・造血器・リンパ系の疾患・病態について病因、疫学、症候、主な検査・診断、治療法を説明できる。,"D-01-04-01-01, D-01-04-01-02, D-01-04-01-03, D..."
...,...,...,...,...
70,人体各器官の正常構造と機能、病態、診断、治療,精神系,精神系の構造と機能について基本的事項を説明できる。,
71,人体各器官の正常構造と機能、病態、診断、治療,精神系,精神系でみられる症状・症候について説明できる。,"D-15-02-na-02,D-15-02-na-02,D-15-02-na-02,D-15..."
72,人体各器官の正常構造と機能、病態、診断、治療,精神系,精神系で行う検査方法について基本的事項を説明できる。,"D-15-01-na-05,,,,,,,,D-15-01-na-02"
73,人体各器官の正常構造と機能、病態、診断、治療,精神系,精神系疾患に特異的な治療法について基本的事項を説明できる。,"D-15-01-na-01,D-15-01-na-03,D-15-01-na-04"
